In [1]:
from floodligt_functions import clean_metadata
from floodligt_functions import read_position_data_jsonl

import numpy as np
from pathlib import Path
import json
import io
import polars as pl
import floodlight
#from floodlight.io.secondspectrum import read_position_data_jsonl
from floodlight.io.secondspectrum import read_teamsheets_from_meta_json
import json
import tempfile
from typing import Union, Tuple, Dict
from pathlib import Path
from floodlight.core.pitch import Pitch
from floodlight.io.utils import get_and_convert
from floodlight.core.teamsheet import Teamsheet
from floodlight.core.xy import XY
from floodlight.core.code import Code
from floodlight.models.kinematics import AccelerationModel
from floodlight.models.kinematics import DistanceModel
from floodlight.models.kinematics import VelocityModel
from floodlight.core.property import PlayerProperty
from floodlight.models.base import BaseModel, requires_fit
from floodlight.models.kinematics import VelocityModel, AccelerationModel
from floodlight.models.kinetics import MetabolicPowerModel


In [2]:
velocity=np.load("np_arrays/velocity.npy")

In [3]:
acceleration=np.load("np_arrays/acceleration.npy")

In [4]:
xy=np.load("np_arrays/xy.npy")

In [2]:
es=np.load("np_arrays/equivalent_slope.npy")

In [6]:
em=np.load("np_arrays/equivalent_mass.npy")

In [15]:
xy

array([[39.94, -0.09,   nan, ...,   nan,   nan,   nan],
       [39.8 , -0.09,   nan, ...,   nan,   nan,   nan],
       [39.67, -0.09,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan]],
      shape=(2901801, 42))

In [17]:
equivalent_slope = MetabolicPowerModel._calc_es(
            velocity, acceleration
        )
np.save("np_arrays/equivalent_slope.npy", equivalent_slope)

In [8]:
equivalent_slope

array([[-0.31403945,         nan, -0.1866439 , ...,         nan,
                nan,         nan],
       [-0.31436369,         nan, -0.09332195, ...,         nan,
                nan,         nan],
       [-0.15534548,         nan, -0.06598513, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]], shape=(2901801, 21))

In [18]:
equivalent_mass = MetabolicPowerModel._calc_em(equivalent_slope)
np.save("np_arrays/equivalent_mass.npy", equivalent_mass)

In [10]:
equivalent_mass

array([[1.04815112,        nan, 1.01726887, ...,        nan,        nan,
               nan],
       [1.04824832,        nan, 1.00434505, ...,        nan,        nan,
               nan],
       [1.01199418,        nan, 1.00217465, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]], shape=(2901801, 21))

In [ ]:
es_power = np.stack(
            (
                np.power(es, 5),
                np.power(es, 4),
                np.power(es, 3),
                np.power(es, 2),
                es,
                np.ones(es.shape),
            ),
            axis=-1,
        )
np.save("np_arrays/es_power.npy", es_power)

In [ ]:
v_trans = MetabolicPowerModel._calc_v_trans(es_power)

: 

: 

In [8]:
running = MetabolicPowerModel._is_running(velocity, equivalent_slope)

: 

In [ ]:
ecl = MetabolicPowerModel._calc_ecl(equivalent_slope, velocity, equivalent_mass, 3.6)

: 

In [9]:
metabolic_power = MetabolicPowerModel._calc_metabolic_power(
            equivalent_slope, velocity, equivalent_mass, 25, 3.6
        )

: 

In [ ]:
equivalent_mass